<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_PowerBI_Import_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install bs4

In [2]:
pip install urllib3

In [3]:
pip install marshmallow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.2 MB/s eta 0:00:00


In [30]:
import requests
import pandas as pd
import math
import ssl
import concurrent.futures
import time

from datetime import datetime, timedelta, date
from typing import Dict, List, Optional, Union
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from urllib3.poolmanager import PoolManager
from urllib3.util.retry import Retry

# Special thanks to https://github.com/burakyilmaz321

class InfoSchema(Schema):
    code = fields.String(data_key="FONKODU", allow_none=True)
    fonunvantip = fields.String(data_key="FONUNVANTIP", allow_none=True)
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        # Convert milliseconds Unix timestamp to date
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        # Fill missing fields with default None
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    class Meta:
        unknown = EXCLUDE

class tefas_get:

    root_url = "https://www.tefas.gov.tr"
    #detail_endpoint = "/api/DB/BindHistoryAllocation"
    info_endpoint = "/api/DB/BindHistoryInfo"
    headers = {
        "Connection": "keep-alive",
        "X-Requested-With": "XMLHttpRequest",
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
        ),
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Origin": "https://www.tefas.gov.tr",
        "Referer": "https://www.tefas.gov.tr/TarihselVeriler.aspx",
    }

    @staticmethod
    def get_combobox_items(url, select_id):
        # Send a GET request to the URL
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code != 200:
            raise Exception(f"Failed to fetch the URL: {response.status_code}")

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the select element by its ID
        select_element = soup.find('select', id=select_id)

        # Check if the select element exists
        if not select_element:
            raise Exception(f"Select element with id '{select_id}' not found")

        # Get all option elements within the select element
        options = select_element.find_all('option')
        options = list(filter(None, options))

        # Extract the text and value for each option
        items = []
        for option in options:
            value = option.get('value')
            items.append(value)

        items.remove('')
        items.remove('Serbest')

        return items

    def fetch_info( self, fonunvantip, start_date_initial, end_date_initial ):

        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial
        range_date = end_date_initial - start_date_initial
        range_interval = 90
        info_schema = InfoSchema(many=True)
        info_result = pd.DataFrame()

        if range_date.days > range_interval :
           counter = range_date.days / range_interval
           counter = math.ceil(counter)
           end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
           counter -= 1

           data = {
                    "fontip": "YAT",
                    "bastarih": self._parse_date(start_date),
                    "bittarih": self._parse_date(end_date),
                    "fonunvantip": fonunvantip,
                    "fonkod": "",
                  }

           info = self._do_post(self.info_endpoint, data)
           info = info_schema.load(info)
           info = pd.DataFrame(info, columns=info_schema.fields.keys())
           info['fonunvantip'] = fonunvantip
           info = info[info['price'] != 0]
           info_result = pd.concat([info_result, info])
           info_result = info_result.reset_index(drop=True)
           info = info.reset_index(drop=True)

           if counter > 0 :
              start_date = end_date + timedelta(days=1)
              end_date = end_date + timedelta(days=range_interval)
              if end_date > end_date_initial :
                 end_date = end_date_initial

        #print(f"Data for {fonunvantip}: {len(info_result)} rows  {start_date_initial} {end_date_initial} counter {counter}" )
        return info_result

    def fetch_info_serial( self, fonunvantips, start_date_initial, end_date_initial ):
        merged = pd.DataFrame()

        for fonunvantip in fonunvantips:
            info = self.fetch_info(fonunvantip, start_date_initial, end_date_initial)
            merged = pd.concat([merged, info])
            #print(f"Data for merged: {fonunvantip} {len(merged)} info {len(info)}" )

        return merged

    def fetch_info_concurrently( self, fonunvantips, start_date_initial, end_date_initial ):
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            merged = pd.DataFrame()
            # Submit all tasks to the executor
            futures = {executor.submit(self.fetch_info, fonunvantip, start_date_initial, end_date_initial): fonunvantip for fonunvantip in fonunvantips}

            # Retrieve results as they complete
            for future in concurrent.futures.as_completed(futures):
                info = future.result()
                merged = pd.concat([merged, info])
                #print(f"Data for merged: {fonunvantip} {len(merged)} info {len(info)}" )

            return merged

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        columns: Optional[List[str]] = None,
		concurrent = False
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")

        merged = pd.DataFrame()
        fonunvantips = self.get_combobox_items(url="https://www.tefas.gov.tr/TarihselVeriler.aspx", select_id="DropDownListFundTypeExplanationYAT")
        if concurrent != False :
            merged = self.fetch_info_concurrently(fonunvantips, start_date_initial, end_date_initial)
        else :
            merged = self.fetch_info_serial(fonunvantips, start_date_initial, end_date_initial)

        merged = merged[columns] if columns and not merged.empty else merged

        return merged

    def _do_post(self, endpoint: str, data: Dict[str, str]) -> Dict[str, str]:
        time.sleep(1)
        session = requests.Session()
        retry_strategy = Retry(
            total=3,  # Total number of retries
            backoff_factor=1,  # Backoff factor for retries
            status_forcelist=[429, 500, 502, 503, 504],  # Retry on these status codes
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)

        self.cookies = session.cookies.get_dict()

        response = session.post(
            url=f"{self.root_url}/{endpoint}",
            data=data,
            cookies=self.cookies,
            headers=self.headers,
        )
        # Check the response status code and content
        if response.status_code != 200:
            print(f"Request failed with status code: {response.status_code}")
            print(f"Response content: {response.text}")
            return {}  # Return an empty dictionary if the request failed
        try:
            return response.json().get("data", {})
        except ValueError as e:
            print(f"Error decoding JSON response: {e}")
            print(f"Response content: {response.text}")
            return {}

    def _parse_date(self, date: Union[str, datetime]) -> str:
        if isinstance(date, datetime):
            formatted = datetime.strftime(date, "%d.%m.%Y")
        elif isinstance(date, str):
            try:
                parsed = datetime.strptime(date, "%Y-%m-%d")
            except ValueError as exc:
                raise ValueError(
                    "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
                ) from exc
            else:
                formatted = datetime.strftime(parsed, "%d.%m.%Y")
        else:
            raise ValueError(
                "`date` should be a string like 'YYYY-MM-DD' "
                "or a `datetime.datetime` object."
            )
        return formatted

def calculate_rsi(group, window_length):
    # Calculate the daily price changes
    group = group.sort_values('date', ascending=True)
    group['price_change'] = group['price'].diff()

    # Separate the gains and losses
    group['gain'] = group['price_change'].apply(lambda x: x if x > 0 else 0)
    group['loss'] = group['price_change'].apply(lambda x: abs(x) if x < 0 else 0)

    # Calculate the average gain and loss using the exponential moving average
    group['avg_gain'] = group['gain'].rolling(window=window_length, min_periods=1).mean()
    group['avg_loss'] = group['loss'].rolling(window=window_length, min_periods=1).mean()

    group = group.reset_index(drop=True)  # This line is added

    group.loc[group['avg_loss'] == 0, 'rs'] = 0
    group.loc[group['avg_loss'] == 0, 'rsi'] = 100

    # Calculate the Relative Strength (RS)
    group.loc[group['avg_loss'] != 0, 'rs'] = group['avg_gain'] / group['avg_loss']

    # Calculate the RSI
    group.loc[group['avg_loss'] != 0, 'rsi'] = 100 - (100 / (1 + group['rs']))

    # Forward fill RSI values to ensure we have RSI for each day
    group['rsi'] = group['rsi'].fillna(method='ffill')

    group.iloc[:window_length, group.columns.get_loc('rs')] = 0
    group.iloc[:window_length, group.columns.get_loc('rsi')] = 0
    group = group.sort_values('date', ascending=False)

    return group

tefas = tefas_get()

time_delta = 366
start_date_calc = date.today() - timedelta(days=time_delta)
date_start = start_date_calc.strftime("%Y-%m-%d")
date_end = date.today().strftime("%Y-%m-%d")

fetched_data = pd.DataFrame()
fetched_data = tefas.fetch(start=date_start, end=date_end, columns=["code", "date", "price", "fonunvantip", "title", "market_cap", "number_of_shares", "number_of_investors" ], concurrent=True)
fetched_data['date'] = pd.to_datetime(fetched_data['date'], errors='coerce')
fetched_data['date'].dt.strftime('%Y-%m-%d')
fetched_data['date'] = fetched_data['date'].dt.date
fetched_data['price'].astype(float,False)
fetched_data[(fetched_data!=0)&(pd.isnull(fetched_data))]

# Group by stock and apply the RSI calculation
window_length = 14
fetched_data = fetched_data.groupby(['code','fonunvantip']).apply(calculate_rsi, window_length=window_length).reset_index(drop=True)

fetched_data = fetched_data.fillna(0)
fetched_data['rsi'].astype(float,False)
fetched_data['avg_loss'].astype(float,False)
fetched_data['avg_gain'].astype(float,False)
fetched_data['gain'].astype(float,False)
fetched_data['loss'].astype(float,False)
fetched_data['price_change'].astype(float,False)
fetched_data['market_cap'].astype(float,False)
fetched_data['number_of_shares'].astype(float,False)
fetched_data['number_of_investors'].astype(float,False)


Error decoding JSON response: Expecting value: line 1 column 1 (char 0)
Response content: <html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: <  891378466802924190><br><br><a href='javascript:history.back();'>[Go Back]</body></html>
Error decoding JSON response: Expecting value: line 1 column 1 (char 0)
Response content: <html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: <  891378466808382616><br><br><a href='javascript:history.back();'>[Go Back]</body></html>
Error decoding JSON response: Expecting value: line 1 column 1 (char 0)
Response content: <html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: <  891378466806445577><br><br><a href='javascript:history.back();'>[Go Back]</body>

0         921.0
1         912.0
2         917.0
3         919.0
4         905.0
          ...  
100684      1.0
100685      1.0
100686      1.0
100687      1.0
100688      1.0
Name: number_of_investors, Length: 100689, dtype: float64

In [32]:
fetched_data = fetched_data.drop(['price_change', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs'], axis=1)
#fetched_data.to_excel("fetched_data.xlsx", index=False)
fetched_data

KeyError: "['price_change', 'gain', 'loss', 'avg_gain', 'avg_loss', 'rs'] not found in axis"

In [38]:
fetched_data_agg = fetched_data.copy()

# Aggregate duplicate 'code' values by taking the mean of 'price'
fetched_data_agg = fetched_data_agg.groupby(['code', 'title', 'fonunvantip'])['price'].mean().reset_index()

# Pivot the temporary DataFrame, filling missing values with False and others with True
fon_table = fetched_data_agg.pivot(index=['code','title'], columns='fonunvantip', values='price').notnull()

del fetched_data_agg

In [39]:
fon_table

,fonunvantip,Agresif Değişken,Alternatif,Altın,Borçlanma Araçları,Dengeli Değişken,Değişken,Diğer Fon Sepeti,Döviz,Döviz Cinsinden İhraç (Dolar),Döviz Cinsinden İhraç (Euro),Çalışanlarına Yönelik,Çoklu Varlık
code,title,,,,,,,,,,,,
AAK,ATA PORTFÖY ÇOKLU VARLIK DEĞİŞKEN FON,False,False,False,False,False,True,False,False,False,False,False,True
ABU,AK PORTFÖY BİREBİR BANKACILIK DEĞİŞKEN ÖZEL FON,False,False,False,False,False,True,False,False,False,False,False,False
AC3,PARDUS PORTFÖY ÜÇÜNCÜ SERBEST (DÖVİZ) FON,False,False,False,False,False,False,False,True,True,False,False,False
AC6,PARDUS PORTFÖY DÖRDÜNCÜ SERBEST (DÖVİZ) FON,False,False,False,False,False,False,False,True,True,False,False,False
ACD,İSTANBUL PORTFÖY İKİNCİ DEĞİŞKEN FON,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZP9,ZİRAAT PORTFÖY AKİK KATILIM SERBEST (DÖVİZ-AVRO) FON,False,False,False,False,False,False,False,True,False,True,False,False
ZPA,ZİRAAT PORTFÖY SERBEST (DÖVİZ) FON,False,False,False,False,False,False,False,True,True,False,False,False
ZPF,ZİRAAT PORTFÖY KATILIM FONU (DÖVİZ),False,False,False,False,False,False,False,True,True,False,False,False


In [44]:
fetched_data_new = fetched_data.copy()

# Drop the 'fonunvantip' column
fetched_data_new = fetched_data_new.drop(['fonunvantip','title'], axis=1)

# Keep only the first record for each repeating 'fonunvantip'
fetched_data_new = fetched_data_new.drop_duplicates(subset=['code','date'], keep='first')

In [45]:
fetched_data_new

,code,date,price,market_cap,number_of_shares,number_of_investors,rsi
0,AAK,2024-04-19,19.931387,1.062191e+08,5329238.0,921.0,82.955624
1,AAK,2024-04-18,19.966552,1.078098e+08,5399519.0,912.0,79.009045
2,AAK,2024-04-17,19.932391,1.106496e+08,5551244.0,917.0,74.388451
3,AAK,2024-04-16,20.019781,1.109457e+08,5541802.0,919.0,77.860876
4,AAK,2024-04-15,20.140762,1.090244e+08,5413122.0,905.0,84.437238
...,...,...,...,...,...,...,...
100620,ZSN,2023-07-31,26.514946,3.436072e+05,12959.0,1.0,0.000000
100621,ZSN,2023-07-28,26.506748,3.435010e+05,12959.0,1.0,0.000000
100622,ZSN,2023-07-27,26.504365,3.434701e+05,12959.0,1.0,0.000000
100623,ZSN,2023-07-26,26.502468,3.434455e+05,12959.0,1.0,0.000000


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
from datetime import datetime

# Bugünün tarihini al
today = datetime.now().strftime('%Y-%m-%d')

# Dosya adını tarihle oluştur
sheet_name = 'TEFAS_' + today

# Yeni bir dosya oluşturun (varsa üzerine yazar)
spreadsheet = gc.create(sheet_name)

In [ ]:
# İlk çalışma sayfasını alın
worksheet = spreadsheet.sheet1

from gspread.exceptions import APIError

try:
    set_with_dataframe(worksheet, fetched_data, include_column_header=True)
    print("DataFrame successfully written to Google Sheets.")
except APIError as e:
    print(f"An API error occurred: {e}")
    # You can add more specific error handling here if needed

An API error occurred: {'code': 500, 'message': 'Internal error encountered.', 'status': 'INTERNAL'}


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Create a new spreadsheet
sh = gc.create('TefasGuncel')

# Open the first worksheet
worksheet = gc.open('TefasGuncel').sheet1

# Clear any existing data in the worksheet
worksheet.clear()

# Convert the DataFrame to a list of lists
data = fetched_data.values.tolist()

# Insert the data into the worksheet in batches to avoid exceeding cell limit
batch_size = 50000  # Adjust this value based on your data and the sheet's existing content
for i in range(0, len(data), batch_size):
    batch = data[i: i + batch_size]
    for row in batch:
        for j in range(len(row)):
            if isinstance(row[j], date):
                row[j] = row[j].strftime('%Y-%m-%d')  # Format date objects as strings
    worksheet.insert_rows(batch) # Insert data in batches
    print(f"Inserted rows {i} to {i + len(batch)}")

Inserted rows 0 to 50000
Inserted rows 50000 to 100000
Inserted rows 100000 to 150000
Inserted rows 150000 to 200000
Inserted rows 200000 to 250000
Inserted rows 250000 to 300000
Inserted rows 300000 to 350000


APIError: {'code': 400, 'message': 'Invalid requests[0].insertDimension: Bu işlem, çalışma kitabındaki hücre sayısını 10000000 hücrelik sınırın üzerine çıkaracaktır.', 'status': 'INVALID_ARGUMENT'}